<a href="https://colab.research.google.com/github/erickuo5124/MLG_HW2/blob/main/LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!ls '/content/drive/MyDrive/MLG/hw2/hw2_data'

dataset1  dataset2  dataset3  README.md


In [34]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

In [35]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/test.csv')
content = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/content.csv', delimiter='\t', header=None, index_col=0)
upload = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/upload.csv')

In [36]:
import torch
import networkx as nx
from torch_geometric.data import Data, ClusterData, ClusterLoader
from torch_geometric.utils import from_networkx

edge_index = torch.tensor([train['from'].values, train['to'].values], dtype=torch.long)
x = torch.tensor(content.sort_index().values, dtype=torch.float)
y = torch.tensor(train['label'].values, dtype=torch.float)
train_data = Data(edge_index=edge_index, x=x, y=y)

In [44]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
  def __init__(self, hidden_layer):
    super(Net, self).__init__()
    torch.manual_seed(12345)
    self.conv1 = GCNConv(train_data.num_features, hidden_layer)
    self.conv2 = GCNConv(hidden_layer, hidden_layer)
    self.conv3 = GCNConv(hidden_layer, 2)
    self.classifier = Linear(2, 1)

  def forward(self, x, edge_index):
    h = self.conv1(x, edge_index)
    h = h.tanh()
    h = self.conv2(h, edge_index)
    h = h.tanh()
    h = self.conv3(h, edge_index)
    h = h.tanh()  # Final GNN embedding space.
    
    # Apply a final (linear) classifier.
    nodes = self.classifier(h)
    nodes = torch.squeeze(nodes)
    ids_from, ids_to = edge_index[0], edge_index[1]
    preds = torch.index_select(nodes, 0, ids_from) - torch.index_select(nodes, 0, ids_to)

    
    return torch.sigmoid(preds)

In [50]:
from sklearn.metrics import roc_auc_score, average_precision_score

model = Net(128)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
  optimizer.zero_grad()
  out = model(train_data.x, train_data.edge_index)
  loss = criterion(out, train_data.y)
  print(out)
  print(f'Epoch: {epoch:03d}, Loss: {loss:.6f}')
  print(f'roc_auc_score: {roc_auc_score(train_data.y.detach().numpy(), out.detach().numpy())}, average_precision_score: {average_precision_score(train_data.y.detach().numpy(), out.detach().numpy())}')
  loss.backward()
  optimizer.step()

tensor([0.4975, 0.5016, 0.5049,  ..., 0.5000, 0.5019, 0.5132],
       grad_fn=<SigmoidBackward>)
Epoch: 000, Loss: 0.693110
roc_auc_score: 0.5090903919180917, average_precision_score: 0.5155856433701128
tensor([0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward>)
Epoch: 001, Loss: 0.693146
roc_auc_score: 0.44294133041179373, average_precision_score: 0.4646062503444549
tensor([0.5001, 0.4999, 0.5000,  ..., 0.5017, 0.4999, 0.5000],
       grad_fn=<SigmoidBackward>)
Epoch: 002, Loss: 0.692955
roc_auc_score: 0.6031052333223479, average_precision_score: 0.5627745296394487
tensor([0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward>)
Epoch: 003, Loss: 0.693147
roc_auc_score: 0.501631383816418, average_precision_score: 0.4988988526319552
tensor([0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
       grad_fn=<SigmoidBackward>)
Epoch: 004, Loss: 0.693147
roc_auc_score: 0.5, average_precision_score: 0.4978125719548699
